In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Load Data
train_df = pd.read_csv('/mnt/data/train_tfidf_features.csv')
test_df = pd.read_csv('/mnt/data/test_tfidf_features.csv')
submission_df = pd.read_csv('/mnt/data/sample_submission.csv')

# Extract features and labels
X = train_df.drop('label', axis=1)
y = train_df['label']

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
models = {
    'Logistic Regression': LogisticRegression(),
    'SVM': SVC(probability=True),
    'Naive Bayes': MultinomialNB(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

# Train and evaluate models
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    y_proba = model.predict_proba(X_val)[:, 1]
    print(f"{name} Performance:")
    print(classification_report(y_val, y_pred))
    print(f"ROC AUC: {roc_auc_score(y_val, y_proba)}")
    print("\n")

# Choose the best model, train on full training data, and make predictions on test set
best_model = LogisticRegression()  # Replace with the best model based on evaluation
best_model.fit(X, y)
test_predictions = best_model.predict(test_df)

# Prepare submission
submission_df['label'] = test_predictions
submission_df.to_csv('submission.csv', index=False)
